## Loading Your the Data


In [5]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [6]:
# load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
pinecone_api_key = os.getenv('PINECODE_API_KEY')

In [7]:
# loading PDF, DOCX and TXT files as LangChain Documents
def load_document(file):
    import os
    name, extension = os.path.splitext(file)

    if extension == '.pdf':
        from langchain.document_loaders import PyPDFLoader
        from langchain_community.document_loaders import PDFPlumberLoader
        print(f'Loading {file}')
        loader = PyPDFLoader(file)
    elif extension == '.docx':
        from langchain.document_loaders import Docx2txtLoader
        print(f'Loading {file}')
        loader = Docx2txtLoader(file)
    elif extension == '.txt':
        from langchain.document_loaders import TextLoader
        loader = TextLoader(file)
    else:
        print('Document format is not supported!')
        return None

    data = loader.load()
    return data
  

In [8]:
def chunk_data(data, chunk_size=256):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)
    chunks = text_splitter.split_documents(data)
    return chunks
    

In [9]:
def print_embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-3-small')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    # check prices here: https://openai.com/pricing
    print(f'Total Tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.00002:.6f}')

In [10]:
# def insert_or_fetch_embeddings(index_name, chunks):
#     # importing the necessary libraries and initializing the Pinecone client
#     import pinecone
#     from langchain_community.vectorstores import Pinecone
#     from langchain_openai import OpenAIEmbeddings
#     from pinecone import ServerlessSpec

    
#     pc = pinecone.Pinecone(pinecone_api_key)
        
#     embeddings = OpenAIEmbeddings(model='text-embedding-3-small', dimensions=1536)  # 512 works as well

#     # loading from existing index
#     if index_name in pc.list_indexes().names():
#         print(f'Index {index_name} already exists. Loading embeddings ... ', end='')
#         vector_store = Pinecone.from_existing_index(index_name, embeddings)
#         print('Ok')
#     else:
#         # creating the index and embedding the chunks into the index 
#         print(f'Creating index {index_name} and embeddings ...', end='')

#         # creating a new index
#         pc.create_index(
#             name=index_name,
#             dimension=1536,
#             metric='cosine',
#             spec=ServerlessSpec(
#                 cloud="aws",
#                 region="us-east-1"
#         ) 
#         )

#         # processing the input documents, generating embeddings using the provided `OpenAIEmbeddings` instance,
#         # inserting the embeddings into the index and returning a new Pinecone vector store object. 
#         vector_store = Pinecone.from_documents(chunks, embeddings, index_name=index_name)
#         print('Ok')
        
#     return vector_store
    

In [11]:
# def delete_pinecone_index(index_name='all'):
#     import pinecone
#     pc = pinecone.Pinecone(pinecone_api_key)
    
#     if index_name == 'all':
#         indexes = pc.list_indexes().names()
#         print('Deleting all indexes ... ')
#         for index in indexes:
#             pc.delete_index(index)
#         print('Ok')
#     else:
#         print(f'Deleting index {index_name} ...', end='')
#         pc.delete_index(index_name)
#         print('Ok')
    

In [12]:
def ask_and_get_answer(vector_store, q, k=3):
    from langchain.chains import RetrievalQA
    from langchain_openai import ChatOpenAI

    llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)

    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': k})

    chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
    
    answer = chain.invoke(q)
    return answer

In [13]:
def create_embeddings_chroma(chunks, persist_directory='./chroma_db'):
    from langchain.vectorstores import Chroma
    from langchain_openai import OpenAIEmbeddings

    # Instantiate an embedding model from OpenAI (smaller version for efficiency)
    embeddings = OpenAIEmbeddings(model='text-embedding-3-small', dimensions=153)  

    # Create a Chroma vector store using the provided text chunks and embedding model, 
    # configuring it to save data to the specified directory 
    vector_store = Chroma.from_documents(chunks, embeddings, persist_directory=persist_directory) 

    return vector_store  # Return the created vector store


In [14]:
def load_embeddings_chroma(persist_directory='./chroma_db'):
    from langchain.vectorstores import Chroma
    from langchain_openai import OpenAIEmbeddings

    # Instantiate the same embedding model used during creation
    embeddings = OpenAIEmbeddings(model='text-embedding-3-small', dimensions=1536) 

    # Load a Chroma vector store from the specified directory, using the provided embedding function
    vector_store = Chroma(persist_directory=persist_directory, embedding_function=embeddings) 

    return vector_store  # Return the loaded vector store


In [16]:
data = load_document("rag_powered_by_google_search.pdf")

# Splitting the document into chunks
chunks = chunk_data(data, chunk_size=256)

# Creating a Chroma vector store using the provided text chunks and embedding model (default is text-embedding-3-small)
vector_store = create_embeddings_chroma(chunks)

Loading rag_powered_by_google_search.pdf


In [17]:
# Asking questions
# q = 'What was googles revenue in q1 of 2024?'
q = 'Using the documnet is this a good company to invest in?'
answer = ask_and_get_answer(vector_store, q)
print(answer)

{'query': 'Using the documnet is this a good company to invest in?', 'result': "I don't have access to specific documents or information about Google's financial performance or investment prospects. To determine if a company is good to invest in, it's recommended to conduct thorough research on their financial statements, competitive position, growth prospects, and other relevant factors. It's always advisable to consult with a financial advisor before making any investment decisions."}


## Adding Memory

In [18]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain  # Import class for building conversational AI chains 
from langchain.memory import ConversationBufferMemory  # Import memory for storing conversation history

# Instantiate a ChatGPT LLM (temperature controls randomness)
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)  

# Configure vector store to act as a retriever (finding similar items, returning top 5)
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 5})  


# Create a memory buffer to track the conversation
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

crc = ConversationalRetrievalChain.from_llm(
    llm=llm,  # Link the ChatGPT LLM
    retriever=retriever,  # Link the vector store based retriever
    memory=memory,  # Link the conversation memory
    chain_type='stuff',  # Specify the chain type
    verbose=False  # Set to True to enable verbose logging for debugging
)


In [19]:
# from langchain_openai import ChatOpenAI
# from langchain.chains import ConversationalRetrievalChain
# from langchain.memory import ConversationBufferMemory

# def create_conversational_retrieval_chain(vector_store):
#     # Instantiate a ChatGPT LLM (temperature controls randomness)
#     llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

#     # Configure vector store to act as a retriever (finding similar items, returning top 5)
#     retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 5})

#     # Create a memory buffer to track the conversation
#     memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

#     # Create the Conversational Retrieval Chain
#     crc = ConversationalRetrievalChain.from_llm(
#         llm=llm,  # Link the ChatGPT LLM
#         retriever=retriever,  # Link the vector store based retriever
#         memory=memory,  # Link the conversation memory
#         chain_type='stuff',  # Specify the chain type
#         verbose=False  # Set to True to enable verbose logging for debugging
#     )

#     return crc

# # Usage example:
# # crc_instance = create_conversational_retrieval_chain(vector_store)


In [20]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

def create_conversational_retrieval_chain(vector_store):
    from langchain_openai import ChatOpenAI
    from langchain.chains import ConversationalRetrievalChain
    from langchain.memory import ConversationBufferMemory

    # Instantiate a ChatGPT LLM (temperature controls randomness)
    llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

    # Configure vector store to act as a retriever (finding similar items, returning top 5)
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 5})

    # Create a memory buffer to track the conversation
    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

    # Create the Conversational Retrieval Chain
    crc = ConversationalRetrievalChain.from_llm(
        llm=llm,  # Link the ChatGPT LLM
        retriever=retriever,  # Link the vector store based retriever
        memory=memory,  # Link the conversation memory
        chain_type='stuff',  # Specify the chain type
        verbose=False  # Set to True to enable verbose logging for debugging
    )

    return crc

def ask_question(q, chain):
    result = chain.invoke({'question': q})
    return result

# Usage example:
# Initialize the chain once
# crc_instance = create_conversational_retrieval_chain(vector_store)

# Ask questions with preserved memory
# answer1 = ask_question(crc_instance, "What is the capital of France?")
# print(answer1)

# answer2 = ask_question(crc_instance, "What is the population of that capital?")
# print(answer2)


In [21]:
data = load_document("rag_powered_by_google_search.pdf")
chunks = chunk_data(data, chunk_size=256)
vector_store = create_embeddings_chroma(chunks)
crc_instance = create_conversational_retrieval_chain(vector_store)

Loading rag_powered_by_google_search.pdf


In [22]:
q = 'What was googles revenue in q1 of 2024?'
result = ask_question(q, crc_instance)
print(result)

{'question': 'What was googles revenue in q1 of 2024?', 'chat_history': [HumanMessage(content='What was googles revenue in q1 of 2024?'), AIMessage(content="Google's revenue in Q1 of 2024 was $80,539 million.")], 'answer': "Google's revenue in Q1 of 2024 was $80,539 million."}


In [23]:
result

{'question': 'What was googles revenue in q1 of 2024?',
 'chat_history': [HumanMessage(content='What was googles revenue in q1 of 2024?'),
  AIMessage(content="Google's revenue in Q1 of 2024 was $80,539 million.")],
 'answer': "Google's revenue in Q1 of 2024 was $80,539 million."}

In [24]:
q = 'Multiply that number by 2.'
result = ask_question(q, crc_instance)
print(result)

{'question': 'Multiply that number by 2.', 'chat_history': [HumanMessage(content='What was googles revenue in q1 of 2024?'), AIMessage(content="Google's revenue in Q1 of 2024 was $80,539 million."), HumanMessage(content='Multiply that number by 2.'), AIMessage(content="Google's revenue in Q1 of 2024 was $69.787 billion. Multiplying this by 2 would result in $139.574 billion.")], 'answer': "Google's revenue in Q1 of 2024 was $69.787 billion. Multiplying this by 2 would result in $139.574 billion."}


In [33]:
for item in result['chat_history']:
    print(item)

content='What was googles revenue in q1 of 2024?'
content="Google's revenue in Q1 of 2024 was $80,539 million."
content='Multiply the revenue number by 2.'
content="Google's revenue in Q1 of 2024 was $69.787 billion. Multiplying this by 2 would result in $139.574 billion."
content='What was googles revenue in q1 of 2024?'
content="Google's revenue in Q1 of 2024 was $80,539 million. When multiplied by 2, the total revenue would be $161,078 million."


### Loop Questions

In [35]:
while True:
    q = input('Your question: ')
    if q.lower() in 'exit quit bye':
        print('Bye bye!')
        break
    result = ask_question(q, crc_instance)
    print(result['answer'])
    print('-' * 100)

This document appears to be related to Alphabet Inc.'s Amended and Restated 2021 Stock Plan, specifically the Form of Alphabet Restricted Stock Unit Agreement.
----------------------------------------------------------------------------------------------------
The key information in this document includes details about Alphabet Inc.'s Amended and Restated 2021 Stock Plan, specifically the Form of Alphabet Restricted Stock Unit Agreement. Additionally, it mentions corporate governance information such as the certificate of incorporation, bylaws, and governance guidelines, which can be found on Google's Keyword blog at https://www.blog.google/.
----------------------------------------------------------------------------------------------------
Bye bye!


## Using a Custom Prompt

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 5})
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


system_template = r'''
Use the following pieces of context to answer the user's question.
Before answering translate your response to Spanish.
If you don't find the answer in the provided context, just respond "I don't know."
---------------
Context: ```{context}```
'''

user_template = '''
Question: ```{question}```
'''

messages= [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template(user_template)
]

qa_prompt = ChatPromptTemplate.from_messages(messages)

crc = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    chain_type='stuff',
    combine_docs_chain_kwargs={'prompt': qa_prompt },
    verbose=True
)